In [3]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import acq
import env

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
    - Includes all the tables in the zillow database.
    - Used LEFT JOIN to keep all columns of the properties_2017 table
    - Used a subquerry to filter DISTINCT property id with 2017 transaction
    - WHERE statement to keep latitude/longitude where IS NOT NULL.

In [4]:
df = acq.get_zillow_data()
df.head(2)

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.00,NaN,NaN,NaN,NaN,1727539,NaN,...,0,0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,261.00,NaN,NaN,NaN,NaN,1387261,NaN,...,1,0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None


In [26]:
pd.set_option("display.max_rows", None, "display.max_columns", None) 

In [ ]:
df

2. Summarize your data

In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,77579.0,1.300828e+07,3.519399e+06,1.071186e+07,1.153820e+07,1.253054e+07,1.421136e+07,1.676893e+08
typeconstructiontypeid,223.0,6.040359e+00,5.560348e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77579.0,2.618245e+02,5.141596e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
heatingorsystemtypeid,49571.0,3.921749e+00,3.594780e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,207.0,7.386473e+00,2.728030e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,25007.0,1.812013e+00,2.965768e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,77579.0,1.495392e+06,8.609686e+05,3.490000e+02,7.521420e+05,1.498227e+06,2.240879e+06,2.982274e+06
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

In [5]:
df.dtypes

parcelid                    int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
                           ...   
buildingclassdesc          object
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
Length: 69, dtype: object

In [6]:
df.shape

(77579, 69)

In [15]:
df.count()

parcelid                  77579
typeconstructiontypeid      223
storytypeid                  50
propertylandusetypeid     77579
heatingorsystemtypeid     49571
                          ...  
buildingclassdesc            15
heatingorsystemdesc       49571
propertylandusedesc       77579
storydesc                    50
typeconstructiondesc        223
Length: 69, dtype: int64

In [20]:
df.isnull().sum()

parcelid                      0
typeconstructiontypeid    77356
storytypeid               77529
propertylandusetypeid         0
heatingorsystemtypeid     28008
                          ...  
buildingclassdesc         77564
heatingorsystemdesc       28008
propertylandusedesc           0
storydesc                 77529
typeconstructiondesc      77356
Length: 69, dtype: int64

In [24]:
def summarize():
    print(df.shape)
    print(df.info())
    print(df.isnull().sum())

In [25]:
summarize()

(77579, 69)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77579 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  fl

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.